<a href="https://colab.research.google.com/github/zirrashaphan/bcd_BASELINE_MODELS_PERFORMANCE_COMPARISON/blob/main/bcd_BASELINE_MODELS_PERFORMANCE_COMPARISON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

**DEFINE COLUMN NAMES FOR DATASET**

In [ ]:
# Define column names for the dataset
cols = ['id', 'diagnosis', 'radius1', 'texture1', 'perimeter1',
       'area1', 'smoothness1', 'compactness1', 'concavity1',
       'concavity_points1', 'symmetry1', 'fractal_dimension1','radius2', 'texture2', 'perimeter2',
       'area2', 'smoothness2', 'compactness2', 'concavity2',
       'concavity_points2', 'symmetry2', 'fractal_dimension2','radius3', 'texture3', 'perimeter3',
       'area3', 'smoothness3', 'compactness3', 'concavity3',
       'concavity_points3', 'symmetry3', 'fractal_dimension3']

# Load dataset
# Dataset: Wisconsin Diagnostic Breast Cancer (WDBC)
df = pd.read_csv('wdbc.data', names=cols)
df.head()

,id,diagnosis,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concavity_points1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concavity_points3,symmetry3,fractal_dimension3
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
# Convert diagnosis column to binary (M=1 for Malignant, B=0 for Benign)
df["diagnosis"].unique()

array(['M', 'B'], dtype=object)

In [ ]:
df["diagnosis"] = (df["diagnosis"] == "M").astype(int)

In [ ]:
df.head()

,id,diagnosis,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concavity_points1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concavity_points3,symmetry3,fractal_dimension3
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
# Data visualization: Plot histograms for each feature grouped by diagnosis
for label in cols[2:]: # Excluding 'id' and 'diagnosis'
  plt.hist(df[df["diagnosis"]==1][label], color='blue', label='M', alpha=0.7, density=True)
  plt.hist(df[df["diagnosis"]==0][label], color='red', label='B', alpha=0.7, density=True)
  plt.title(label)
  plt.ylabel("Probability")
  plt.xlabel(label)
  plt.legend()
  plt.show()

#Train, Validation, Test Datasets

In [ ]:
# Split data into training (60%), validation (20%), and test (20%) sets
train, validate, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])

In [ ]:
def scale_dataset(dataframe, oversample=False):
    """
    Scales feature values using StandardScaler and optionally applies oversampling to balance classes.
    """
    X = dataframe[dataframe.columns[2:]].values # Select feature columns
    y = dataframe[dataframe.columns[1]].values # Select target column

    scaler = StandardScaler()
    X = scaler.fit_transform(X) # Standardize features

    if oversample:
        ros = RandomOverSampler()
        X, y = ros.fit_resample(X, y) # Oversample minority class

    data = np.hstack((X, np.reshape(y,  (-1, 1)))) # Combine features and labels
    return data, X, y

In [ ]:
# Scale datasets and apply oversampling to training set
train, X_train, y_train = scale_dataset(train, oversample=True)
test, X_test, y_test = scale_dataset(test, oversample=False)
validate, X_validate, y_validate = scale_dataset(validate, oversample=False)

#K- Nearest Neighbor

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
# K-Nearest Neighbors (KNN) Classifier
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
y_pred = knn_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.97      0.91      0.94        78
           1       0.83      0.94      0.88        36

    accuracy                           0.92       114
   macro avg       0.90      0.93      0.91       114
weighted avg       0.93      0.92      0.92       114



#Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Naïve Bayes Classifier
nb_model = GaussianNB()
nb_model = nb_model.fit(X_train, y_train)

In [ ]:
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.85      0.90        78
           1       0.74      0.94      0.83        36

    accuracy                           0.88       114
   macro avg       0.85      0.90      0.87       114
weighted avg       0.90      0.88      0.88       114



#Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Logistic Regression Classifier
lg_model = LogisticRegression()
lg_model = lg_model.fit(X_train, y_train)

In [ ]:
y_pred = lg_model.predict(X_test)
print(classification_report(y_test, y_pred))
#

              precision    recall  f1-score   support

           0       0.99      0.91      0.95        78
           1       0.83      0.97      0.90        36

    accuracy                           0.93       114
   macro avg       0.91      0.94      0.92       114
weighted avg       0.94      0.93      0.93       114



#Support Vector Machine


In [ ]:
from sklearn.svm import SVC

In [ ]:
# Support Vector Machine (SVM) Classifier
svm_model = SVC()
svm_model = svm_model.fit(X_train, y_train)

In [ ]:
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.94      0.96        78
           1       0.88      0.97      0.92        36

    accuracy                           0.95       114
   macro avg       0.93      0.95      0.94       114
weighted avg       0.95      0.95      0.95       114

